# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse           Shipments  \
0  Dec  8 2022  3:54PM  252676        10          0086235271   
1  Dec  8 2022  3:54PM  252676        10          0086235281   
2  Dec  8 2022  3:54PM  252676        10          0086235306   
3  Dec  8 2022  3:54PM  252676        10          0086235307   
4  Dec  8 2022  3:54PM  252676        10          0086235308   
5  Dec  8 2022  3:54PM  252676        10          0086235319   
6  Dec  8 2022  3:54PM  252676        10          0086235321   
7  Dec  8 2022  3:54PM  252676        10          0086235320   
8  Dec  8 2022  3:45PM  252675        10        SONSB0002047   
9  Dec  8 2022  3:37PM  252674        19  TR-RN-12122022-BTL   

                        Customer ShipmentStatus  
0              ISDIN Corporation       Released  
1              ISDIN Corporation       Released  
2              ISDIN Corporation       Released  
3              ISDIN Corporation       Released  
4              ISDIN Corporation       Released  
5              ISDIN Corporation       Released  
6              ISDIN Corporation       Released  
7              ISDIN Corporation       Released  
8  Nextsource Biotechnology, LLC       Released  
9         GCH Granules USA, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
57  252672       Released         23
58  252673       Released          1
59  252674       Released          1
60  252675       Released          1
61  252676       Released          8

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Cancelled  Completed  Executing  Released
id                                                       
252672                NaN        NaN        NaN      23.0
252673                NaN        NaN        NaN       1.0
252674                NaN        NaN        NaN       1.0
252675                NaN        NaN        NaN       1.0
252676                NaN        NaN        NaN       8.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Cancelled  Completed  Executing  Released
id                                                       
252572                0.0        0.0        0.0       1.0
252573                0.0        0.0        0.0       1.0
252574                0.0        0.0        0.0       1.0
252575                0.0        2.0        0.0       1.0
252577                1.0        0.0        0.0       0.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Cancelled  Completed  Executing  Released
id                                                       
252572                0.0          0          0         1
252573                0.0          0          0         1
252574                0.0          0          0         1
252575                0.0          2          0         1
252577                1.0          0          0         0

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Cancelled  Completed  Executing  Released
0               252572        0.0          0          0         1
1               252573        0.0          0          0         1
2               252574        0.0          0          0         1
3               252575        0.0          2          0         1
4               252577        1.0          0          0         0

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id  Cancelled Completed Executing Released
0               252572        0.0                            1
1               252573        0.0                            1
2               252574        0.0                            1
3               252575        0.0         2                  1
4               252577        1.0

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                       Customer
0   Dec  8 2022  3:54PM  252676        10              ISDIN Corporation
8   Dec  8 2022  3:45PM  252675        10  Nextsource Biotechnology, LLC
9   Dec  8 2022  3:37PM  252674        19         GCH Granules USA, Inc.
10  Dec  8 2022  3:27PM  252673        19        GUSA Granules USA, Inc.
11  Dec  8 2022  3:11PM  252672        10                        Bio-PRF
34  Dec  8 2022  3:08PM  252642        10                        Bio-PRF
41  Dec  8 2022  2:52PM  252670        19         GCH Granules USA, Inc.
42  Dec  8 2022  2:44PM  252666        10              ISDIN Corporation
51  Dec  8 2022  2:35PM  252665        21              NBTY Global, Inc.
52  Dec  8 2022  2:28PM  252663        21              NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                       Customer  \
0  Dec  8 2022  3:54PM  252676        10              ISDIN Corporation   
1  Dec  8 2022  3:45PM  252675        10  Nextsource Biotechnology, LLC   
2  Dec  8 2022  3:37PM  252674        19         GCH Granules USA, Inc.   
3  Dec  8 2022  3:27PM  252673        19        GUSA Granules USA, Inc.   
4  Dec  8 2022  3:11PM  252672        10                        Bio-PRF   
5  Dec  8 2022  3:08PM  252642        10                        Bio-PRF   
6  Dec  8 2022  2:52PM  252670        19         GCH Granules USA, Inc.   
7  Dec  8 2022  2:44PM  252666        10              ISDIN Corporation   
8  Dec  8 2022  2:35PM  252665        21              NBTY Global, Inc.   
9  Dec  8 2022  2:28PM  252663        21              NBTY Global, Inc.   

   Cancelled Completed Executing Released  
0        0.0                            8  
1        0.0                            1  
2        0.0                            1  
3        0.0                            1  
4        0.0                           23  
5        0.0                            7  
6        0.0                            1  
7        0.0                   1        8  
8        0.0                            1  
9        0.0                            1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                       Customer  \
0  Dec  8 2022  3:54PM  252676        10              ISDIN Corporation   
1  Dec  8 2022  3:45PM  252675        10  Nextsource Biotechnology, LLC   
2  Dec  8 2022  3:37PM  252674        19         GCH Granules USA, Inc.   
3  Dec  8 2022  3:27PM  252673        19        GUSA Granules USA, Inc.   
4  Dec  8 2022  3:11PM  252672        10                        Bio-PRF   
5  Dec  8 2022  3:08PM  252642        10                        Bio-PRF   
6  Dec  8 2022  2:52PM  252670        19         GCH Granules USA, Inc.   
7  Dec  8 2022  2:44PM  252666        10              ISDIN Corporation   
8  Dec  8 2022  2:35PM  252665        21              NBTY Global, Inc.   
9  Dec  8 2022  2:28PM  252663        21              NBTY Global, Inc.   

  Released Executing Completed  
0        8                      
1        1                      
2        1                      
3        1                      
4       23                      
5        7                      
6        1                      
7        8         1            
8        1                      
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Dec  8 2022  3:54PM  252676        10              ISDIN Corporation   
1  Dec  8 2022  3:45PM  252675        10  Nextsource Biotechnology, LLC   
2  Dec  8 2022  3:37PM  252674        19         GCH Granules USA, Inc.   
3  Dec  8 2022  3:27PM  252673        19        GUSA Granules USA, Inc.   
4  Dec  8 2022  3:11PM  252672        10                        Bio-PRF   

  Released Executing Completed  
0        8                      
1        1                      
2        1                      
3        1                      
4       23

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

Date      id Warehouse                       Customer  \
0  Dec  8 2022  3:54PM  252676        10              ISDIN Corporation   
1  Dec  8 2022  3:45PM  252675        10  Nextsource Biotechnology, LLC   
2  Dec  8 2022  3:37PM  252674        19         GCH Granules USA, Inc.   
3  Dec  8 2022  3:27PM  252673        19        GUSA Granules USA, Inc.   
4  Dec  8 2022  3:11PM  252672        10                        Bio-PRF   

   Released  Executing  Completed  
0       8.0        NaN        NaN  
1       1.0        NaN        NaN  
2       1.0        NaN        NaN  
3       1.0        NaN        NaN  
4      23.0        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                       Customer  \
0  Dec  8 2022  3:54PM  252676        10              ISDIN Corporation   
1  Dec  8 2022  3:45PM  252675        10  Nextsource Biotechnology, LLC   
2  Dec  8 2022  3:37PM  252674        19         GCH Granules USA, Inc.   
3  Dec  8 2022  3:27PM  252673        19        GUSA Granules USA, Inc.   
4  Dec  8 2022  3:11PM  252672        10                        Bio-PRF   

   Released  Executing  Completed  
0       8.0        0.0        0.0  
1       1.0        0.0        0.0  
2       1.0        0.0        0.0  
3       1.0        0.0        0.0  
4      23.0        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         4799944     171.0        3.0        2.0
15          252605       0.0        3.0        0.0
16          505159       2.0        0.0        0.0
19         1768545       6.0        5.0        0.0
20          757825       3.0        0.0        2.0
21         6315779      25.0        0.0        0.0
22          252583       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  4799944     171.0        3.0        2.0
1        15   252605       0.0        3.0        0.0
2        16   505159       2.0        0.0        0.0
3        19  1768545       6.0        5.0        0.0
4        20   757825       3.0        0.0        2.0
5        21  6315779      25.0        0.0        0.0
6        22   252583       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     171.0        3.0        2.0
1        15       0.0        3.0        0.0
2        16       2.0        0.0        0.0
3        19       6.0        5.0        0.0
4        20       3.0        0.0        2.0
5        21      25.0        0.0        0.0
6        22       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  171.0
1        15  Released    0.0
2        16  Released    2.0
3        19  Released    6.0
4        20  Released    3.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   15   16   19   20    21   22
Status                                         
Completed    2.0  0.0  0.0  0.0  2.0   0.0  0.0
Executing    3.0  3.0  0.0  5.0  0.0   0.0  0.0
Released   171.0  0.0  2.0  6.0  3.0  25.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   15   16   19   20    21   22
0          Completed    2.0  0.0  0.0  0.0  2.0   0.0  0.0
1          Executing    3.0  3.0  0.0  5.0  0.0   0.0  0.0
2           Released  171.0  0.0  2.0  6.0  3.0  25.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   15   16   19   20    21   22
0  Completed    2.0  0.0  0.0  0.0  2.0   0.0  0.0
1  Executing    3.0  3.0  0.0  5.0  0.0   0.0  0.0
2   Released  171.0  0.0  2.0  6.0  3.0  25.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()